In [ ]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
from data_generator import DataGenerator
from tec_an import TecAn
import numpy as np
from data_util import *
import threading
import multiprocessing
from data_agent import DataAgent, TacProcess
from sklearn.model_selection import train_test_split

In [ ]:
sourceDataGenerator = SourceDataGenerator()

In [ ]:
%%time

#dirs = ["omgusd", "btceur", "btcusd", "ethusd", "ltcusd", "btcusd17", "btcusdAug19"]
dirs = ["omgusd", "btceur", "btcusd", "ethusd", "ltcusd"]
#dirs = ["omgusd"]

#sets = load_datasets(dirs, '2Min')
#save(conc_sets(sets))

In [ ]:
%%time

dirs = ["omgusd", "btceur", "btcusd17", "btcusd", "ethusd", "ltcusd", "btcusdAug19"]
#dirs = ["omgusd", "btceur", "btcusd", "ethusd", "ltcusd"]
#dirs = ["omgusd", "4jul21"]

sets = sourceDataGenerator.load_simple_datasets(dirs, '2Min')


In [ ]:
sourceDataGenerator.save(conc_simple_sets(sets), prefix = "simple_full_")

In [ ]:
vals = ["backtest", "4jul21"]

#dirs = ["omgusd", "4jul21"]

load_simple_datasets(vals, '2Min')


In [ ]:

steps = 1

windows = [5, 10, 20, 30, 40]

windows = [5, 10, 20, 30, 40]


In [ ]:

def test_windows(minutes):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = 1
        )
        for rank in result:
            result['window'] = win
            result['minutes'] = minutes
            
        results.append(result)
    return results
    